<a href="https://colab.research.google.com/github/MOIPA/AiCourse/blob/master/19%E7%9B%AE%E6%A0%87%E6%A3%80%E6%B5%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>